# <span style="color:red;"> Detección de estado </span>

## <span style="color:orange;"> Comprobación de uno de los procesos </span>

Esta función comprueba si el proceso tiene suficientes recursos para acabar, si los tiene supone que en algún momento podrá acabar y devolver todos sus recursos.

En el retorno se devuelve el nuevo vector de disponibles y el nuevo vector de peticiones pendientes con "guiones" para marcar que ya ha sido procesado.

In [73]:
def mark_line(availables,proc_current,proc_requested):
    avail_new = []
    requested_new = []
    # La siguiente condición es para prevenir el que
    # se intente marcar uno ya marcado
    # Sólo es para facilitar la programación
    if proc_requested[0] is "-":
        return False, availables, proc_requested, proc_current
    
    for avail, pend in zip(availables, proc_requested):
        avail_new.append(avail+pend)
        if avail < pend:
            return False, availables, proc_requested, proc_current
    current_new = ["-"]*len(proc_current)
    requested_new = ["-"]*len(proc_requested)
    return True, avail_new, current_new, requested_new

## <span style="color:orange;"> Cálculo del vector de recursos disponibles </span>

Por coherencia entre las matrices definidas, la suma de las veces que se ha asignado un recurso a cada uno de los procesos con los procesos disponibles tiene que dar como resultado el número de recursos totales, es decir: $$\sum C_{ij} + A_j = E_j$$ o lo que es lo mismo para esta función que vamos a crear: $$A_j = E_j - \sum C_{ij}$$

In [74]:
def availables_calc(existent, current):
    avail = existent[:]

    for proc in current:
        for i in range(len(existent)):
            avail[i] -= proc[i]
            if avail[i] < 0:
                return False, avail
    return True, avail

## <span style="color:orange;"> Busca de un proceso para marcar </span>

Esta función busca un proceso para marcar. Puede ser cualquiera de los procesos pero lo importante es que se cumpla que haya suficientes recursos disponibles ($A$) de cada tipo para asignarle todos los que aún le faltan a ese proceso y que están en la correspondiente fila de la matriz de solicitudes ($R_i$)

In [80]:
def search_mark(available,current,requested):
    for i in range(len(current)-1,-1,-1):
        found, available, current[i], requested[i] = mark_line(available, current[i], requested[i])
        if found is True:
            return found , available, current, requested
    return found, available, current, requested

## <span style="color:orange;"> Analiza el estado </span>

Esta función busca busca de forma reiterada procesos para marcar. Si la función de marcado no puede marcar ninguno se comprueba que el número de procesos marcados sea igual al número de procesos totales. Si no se han conseguido marcar todos entonces el estado es inseguro porque no hay ninguna forma de asignar los recursos para que todos los procesos puedan terminar. Si no hay ninguna liberación de recursos por parte de algún proceso el estado inseguro llevará inexoráblemente a un interbloqueo. Si se han marcado todos los procesos entonces el estado es seguro porque existe alguna forma de asignar los recursos de tal forma que todos los procesos puedan acabar.

In [81]:
SECURE_STATE = "Secure"
INSECURE_STATE = "Insecure"
IMPOSSIBLE_PROBLEM = "Imposible"

def state(existent,current, requested):
    pos, available = availables_calc(existent, current)
    if pos is False:
        return IMPOSSIBLE_PROBLEM

    count = 0
    n_proc = len(current)
    found = True
    while found:
        found , available, current, requested = search_mark(available, current, requested)
        if found: 
            count += 1
            print("\nTry: " + str(count))
            print("A = ", available)
            print("C = ", current)
            print("R = ", requested) 
            
    if count == n_proc:
        return SECURE_STATE
    return INSECURE_STATE

## <span style="color:orange;"> Probando </span>

In [82]:
existent = [5,3,2]

current =[
    [1,1,0],
    [2,0,0],
    [1,0,0]
]

requested = [
    [0,0,1],
    [0,1,1],
    [1,1,0],
]

print("\n\n" +state(existent, current, requested)+"\n\n")


Try: 1
A =  [2, 3, 2]
C =  [[1, 1, 0], [2, 0, 0], ['-', '-', '-']]
R =  [[0, 0, 1], [0, 1, 1], ['-', '-', '-']]

Try: 2
A =  [2, 4, 3]
C =  [[1, 1, 0], ['-', '-', '-'], ['-', '-', '-']]
R =  [[0, 0, 1], ['-', '-', '-'], ['-', '-', '-']]

Try: 3
A =  [2, 4, 4]
C =  [['-', '-', '-'], ['-', '-', '-'], ['-', '-', '-']]
R =  [['-', '-', '-'], ['-', '-', '-'], ['-', '-', '-']]


Secure




## <span style="color:red;"> Algoritmo del banquero </span>

Todo lo anterior es simplemente un algoritmo para detectar el estado del sistema en un determinado instante.

El algoritmo del banquero es un concepto dinámico. Este algoritmo consiste en que, ante una petición de un recurso de un proceso se realiza el siguiente proceso:
1. Ante una petición de asignación:
    1. Se hace como si se asignase el recurso, disminuyendo el correspondiente elemento en la matriz de peticiones
    2. Se analiza el estado con el recurtso asignado.
        1. Si es seguro se asigna realmente y se cambian todas las matrices de verdad
        2. Si es inseguro no se asigna, se deja las matrices como estaban iniciales y se mete al proceso con su solicitud en una lista de bloqueados
2. Ante una solicitud de liberación
    1. Se libera el correspondiente recurso de la matriz de recursos asignados que pasará a estar disponible
    2. Se comprueba (por orden de bloqueo) si la solicitud que bloqueó a los procesos ahora es segura o no. Si es segura se realiza y se sigue desbloqueando a todos los procesos de la cola que no introduzcan al sistema en un estado inseguro al ser desbloqueados. Para ello se realiza el mismo proceso que ante cualquier petición de asignación (ver punto 1).
    
Es muy importante no confundir el algoritmo del banquero con el algoritmo de detección explicado anteriormente. El algoritmo del banquero utiliza el algoritmo de detección pero para cada solicitud de recurso.